## Práctica 3: Ecosistema Spark

### Objetivo:
#### Realizar una tarea de manipulación de datos en Spark Identificar las operaciones internas relacionadas con esta tarea.

In [1]:
# imports necesarios
import pandas as pd

In [2]:
# Carga de los datos trás extraerlos y generar el archivo parquet
censo = spark.read.format('parquet').load('data/censo-2011.parquet')

In [3]:
# vemos el esquema de los datos cargados, con el tipo de dato de cada columna
censo.printSchema()

root
 |-- CPRO: string (nullable = true)
 |-- CMUN: string (nullable = true)
 |-- IDHUECO: string (nullable = true)
 |-- NORDEN: string (nullable = true)
 |-- FACTOR: string (nullable = true)
 |-- MNAC: string (nullable = true)
 |-- ANAC: string (nullable = true)
 |-- EDAD: string (nullable = true)
 |-- SEXO: string (nullable = true)
 |-- NACI: string (nullable = true)
 |-- CPAISN: string (nullable = true)
 |-- CPRON: string (nullable = true)
 |-- CMUNN: string (nullable = true)
 |-- ANORES: string (nullable = true)
 |-- ANOM: string (nullable = true)
 |-- ANOC: string (nullable = true)
 |-- ANOE: string (nullable = true)
 |-- CLPAIS: string (nullable = true)
 |-- CLPRO: string (nullable = true)
 |-- CLMUNP: string (nullable = true)
 |-- RES_ANTERIOR: string (nullable = true)
 |-- CPAISUNANO: string (nullable = true)
 |-- CPROUNANO: string (nullable = true)
 |-- CMUNANO: string (nullable = true)
 |-- RES_UNANO: string (nullable = true)
 |-- CPAISDANO: string (nullable = true)
 |-- CPRO

In [4]:
# mostramos número de columnas y número de filas
print("Número de columnas:", len(censo.columns), "\nNúmero de filas:", censo.count())

Número de columnas: 143 
Número de filas: 208751


In [5]:
# Veamos una muestra
with pd.option_context('max_columns', 999):
    sample = censo.limit(10).toPandas()

sample

,CPRO,CMUN,IDHUECO,NORDEN,FACTOR,MNAC,ANAC,EDAD,SEXO,NACI,...,SITPCON,TIPONUC,TAMNUC,NHIJO,NHIJOC,FAMNUM,TIPOPARECIV,TIPOPARSEX,DIFEDAD,NOMBRE_MUN
0,01,059,0000000356,0002,011.0002419100,02,1951,060,1,108,...,4,2,02,1,1,01,01,01,05,Vitoria-Gasteiz
1,01,059,0000001763,0001,013.9078151930,01,1925,086,1,108,...,8,1,01,0,0,01,01,01,06,Vitoria-Gasteiz
2,01,059,0000003332,0002,013.8891865280,07,1928,083,6,108,...,8,1,01,0,0,01,01,01,04,Vitoria-Gasteiz
3,01,059,0000005472,0002,012.1405619400,08,1938,073,6,108,...,8,1,01,0,0,01,01,01,06,Vitoria-Gasteiz
4,01,059,0000009826,0001,012.8972371580,01,1935,076,1,108,...,8,1,01,0,0,01,01,01,03,Vitoria-Gasteiz
5,01,059,0000009827,0001,013.8613513540,01,1935,076,1,108,...,,,,,,,,,,Vitoria-Gasteiz
6,01,059,0000010969,0001,012.6561832520,01,1936,075,1,108,...,8,1,01,0,0,01,01,01,06,Vitoria-Gasteiz
7,01,059,0000017443,0001,021.4528826850,01,1942,069,1,108,...,8,1,01,0,0,01,01,01,07,Vitoria-Gasteiz
8,01,059,0000018682,0002,013.3026470670,10,1952,059,6,108,...,8,1,01,0,0,01,01,01,06,Vitoria-Gasteiz
9,01,059,0000020199,0002,014.1364482280,03,1936,075,6,108,...,8,1,01,0,0,01,01,01,02,Vitoria-Gasteiz


#### Realice un filtrado de las muestras, quedando sólo con la de las personas cuyo país de nacimiento sea Francia o Portugal (nota: el fichero codes-cpais.csv contiene los códigos de país, y el fichero Excel permite identificar el campo que define el país de nacimiento).

In [6]:
# Viendo la excel el código que nos interesa para identificar el país de nacimiento sería 
# CPAISN. Cargamos el csv con los códigos de país e identificamos los correspondientes a 
# los países de Francia y Portugal.
df_codpaises = spark.read.format("csv").option("header", "true").option("sep", ",").load("data/codes-cpais.csv")
df_codpaises.printSchema()
df_codpaises.filter((df_codpaises['LITERAL_PAIS'] == 'FRANCIA') | (df_codpaises['LITERAL_PAIS'] == 'PORTUGAL')).show()

root
 |-- COD_PAIS: string (nullable = true)
 |-- LITERAL_PAIS: string (nullable = true)

+--------+------------+
|COD_PAIS|LITERAL_PAIS|
+--------+------------+
|     110|     FRANCIA|
|     123|    PORTUGAL|
+--------+------------+



In [7]:
# Ya sabemos que los códigos por los que filtrar serán el 110 y el 123, y la columna para
# realizar el filtrado es CPAISN. Y mostramos el número de registros que tendríamos.
censo_filtrado = censo.filter((censo.CPAISN == '110') | (censo.CPAISN == '123'))
print("Número de filas:", censo_filtrado.count())

Número de filas: 1013


In [8]:
# Hacemos un groupBy por la columna filtrada, y un recuento para ver que efectivamente los 
# dos únicos valores tras filtrar son los códigos correspondientes a Francia y Portugal, y 
# obtenemos el número de registros de cada país.
censo_filtrado.groupBy("CPAISN").count().show()

+------+-----+
|CPAISN|count|
+------+-----+
|   110|  787|
|   123|  226|
+------+-----+



#### De las muestras que han quedado, agrúpelas por provincia de residencia y cuente cuántas personas hay en cada provincia.

In [9]:
# Entendemos que el código de provincia será la columna CPRO. Agrupamos por código y guardamos
# en un DF los distintos códigos de provincia y el recuento de registros para cada una.
df_agrup_prov = censo_filtrado.groupBy('CPRO').count()
df_agrup_prov.show(df_agrup_prov.count())

+----+-----+
|CPRO|count|
+----+-----+
|  07|   33|
|  51|    2|
|  15|   28|
|  11|   29|
|  29|   45|
|  42|    1|
|  30|   57|
|  01|    8|
|  22|    1|
|  28|  166|
|  16|    1|
|  35|    9|
|  52|    2|
|  47|   13|
|  43|    8|
|  31|   11|
|  18|   13|
|  27|    6|
|  17|   17|
|  26|    2|
|  09|    2|
|  46|   88|
|  05|    3|
|  19|    4|
|  23|    1|
|  41|   25|
|  08|  156|
|  03|   58|
|  38|    4|
|  40|    2|
|  25|    3|
|  02|    1|
|  44|    2|
|  33|   21|
|  06|    6|
|  48|   13|
|  24|   21|
|  32|    5|
|  20|   14|
|  36|   39|
|  10|    1|
|  37|    7|
|  49|    3|
|  39|    9|
|  12|   15|
|  04|   13|
|  13|    2|
|  14|   11|
|  21|    8|
|  50|   19|
|  45|    5|
+----+-----+



#### Calcule como resultado final la lista de las 10 provincias(con su nombre) con mayor número de personas procedentes de Francia o Portugal (suma de ambos), y ordénela de mayor a menor.

In [10]:
# cargamos csv con los nombres de provincias
df_provincias = spark.read.format("csv").option("header", "true").option("sep", ",").load("data/codes-prov.csv")
df_provincias.printSchema()
df_provincias.show()

root
 |-- CPRO: string (nullable = true)
 |-- NOMBRE_PRO: string (nullable = true)

+----+--------------------+
|CPRO|          NOMBRE_PRO|
+----+--------------------+
|  01|               Alava|
|  02|            Albacete|
|  03|            Alicante|
|  04|             Almería|
|  05|               Avila|
|  06|             Badajoz|
|  07|            Baleares|
|  08|           Barcelona|
|  09|              Burgos|
|  10|             Cáceres|
|  11|               Cádiz|
|  12|Castellón de la P...|
|  13|         Ciudad Real|
|  14|             Córdoba|
|  15|          Coruña, La|
|  16|              Cuenca|
|  17|              Girona|
|  18|             Granada|
|  19|         Guadalajara|
|  20|           Guipúzcoa|
+----+--------------------+
only showing top 20 rows



In [11]:
# con el DF donde tenemos código de provincia y recuento, obtenemos las 10 provincias con 
# más registros.
df_prov_top = df_agrup_prov.orderBy('count', ascending=False).limit(10)
df_prov_top.show()

+----+-----+
|CPRO|count|
+----+-----+
|  28|  166|
|  08|  156|
|  46|   88|
|  03|   58|
|  30|   57|
|  29|   45|
|  36|   39|
|  07|   33|
|  11|   29|
|  15|   28|
+----+-----+



In [12]:
# hacemos join de ambos DFs por la columna de código de provincia
cond = [df_prov_top.CPRO == df_provincias.CPRO]
df_join = df_prov_top.join(df_provincias, cond).drop(df_provincias.CPRO).orderBy('count', ascending=False)
df_join.show()

+----+-----+----------+
|CPRO|count|NOMBRE_PRO|
+----+-----+----------+
|  28|  166|    Madrid|
|  08|  156| Barcelona|
|  46|   88|  Valencia|
|  03|   58|  Alicante|
|  30|   57|    Murcia|
|  29|   45|    Málaga|
|  36|   39|Pontevedra|
|  07|   33|  Baleares|
|  11|   29|     Cádiz|
|  15|   28|Coruña, La|
+----+-----+----------+



#### Recupere esa lista ordenada de 10 provincias junto con el número de inmigrantes obtenido, para mostrarla en pantalla.

In [13]:
# nos quedamos con las dos columnas que nos interesan
df_final = df_join.select("NOMBRE_PRO","count")

In [14]:
# usamos panda para una mejor visualización, y cambiamos el nombre de las columnas
df_final.toPandas().rename(columns={'NOMBRE_PRO':'Provincia','count':'Número de inmigrantes'})

,Provincia,Número de inmigrantes
0,Madrid,166
1,Barcelona,156
2,Valencia,88
3,Alicante,58
4,Murcia,57
5,Málaga,45
6,Pontevedra,39
7,Baleares,33
8,Cádiz,29
9,"Coruña, La",28
